In [4]:
from datetime import datetime,timedelta
import numpy as np
import json
import csv
from datetime import datetime,timedelta
import numpy as np
import json
import csv
import random
import time
from collections import Counter
import pandas
import os
from pathlib import Path
from datetime import datetime
import re

l=0
readf="cabspottingdata/_cabs.txt"
file= open(readf, "r")
text=file.readlines()  
DMT='%Y-%m-%d %H:%M:%S'
from matplotlib import pyplot as plt
plt.rcParams["figure.autolayout"] = True
plt.figure()
manager = plt.get_current_fig_manager()
manager.full_screen_toggle()
plt.show()
plt.rcParams["figure.figsize"] = [17.00, 7.00]
listofcab=[]
for i in range(len(text)):
    listofcab.append('new_'+text[i].split("\"")[1]+'.txt')
for i in range(536):
    readf="cabspottingdata/"+listofcab[i]
    file= open(readf, "r")
    text=file.readlines() 
    writef="SAN1/"+str(i+1)+".txt"
    wfile= open(writef, "w")
    for j in range(len(text)):
        EL=text[j].split(' ')
        wfile.write(EL[0]+","+EL[1]+","+EL[2]+","+datetime.fromtimestamp(int(EL[3].translate({ord('\n'): None}))).strftime('%Y-%m-%d %H:%M:%S')+"\n")
    print(i)
    file.close()
    wfile.close()

IN_DIR = Path("SAN1")
OUT_DIR = Path("SAN_processed")

def parse_timestamp(ts_raw: str):
    ts_raw = ts_raw.strip()
    fmts = [
        "%Y-%m-%d %H:%M:%S",
        "%Y/%m/%d %H:%M:%S",
        "%Y-%m-%dT%H:%M:%S",
    ]
    for f in fmts:
        try:
            return datetime.strptime(ts_raw, f)
        except ValueError:
            pass
    ts_clean = re.sub(r"[TZ]", " ", ts_raw)
    ts_clean = ts_clean.split("+")[0].split(".")[0].strip()
    for f in fmts:
        try:
            return datetime.strptime(ts_clean, f)
        except ValueError:
            pass
    raise ValueError(f"Unrecognized timestamp format: {ts_raw!r}")

def infer_vehicle_id(stem: str):
    if stem.isdigit():
        return stem
    m = re.search(r"\d+", stem)
    return m.group(0) if m else stem

def process_file(in_path: Path, out_path: Path):
    vehicle_id = infer_vehicle_id(in_path.stem)
    rows = []
    total = 0
    bad  = 0

    with in_path.open("r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            total += 1
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            parts = [p.strip() for p in line.split(",")]
            if len(parts) < 4:
                bad += 1
                continue
            try:
                lat = float(parts[0])
                lon = float(parts[1])
                ts_str = parts[-1]
                ts = parse_timestamp(ts_str)
                rows.append((ts, lon, lat))
            except Exception:
                bad += 1
                continue

    rows.sort(key=lambda x: x[0])  # sort by time ascending
    out_path.parent.mkdir(parents=True, exist_ok=True)
    with out_path.open("w", encoding="utf-8") as w:
        for ts, lon, lat in rows:
            w.write(f"{vehicle_id},{ts.strftime('%Y-%m-%d %H:%M:%S')},{lon:.6f},{lat:.6f}\n")

    return total, len(rows), bad
files = list(IN_DIR.rglob("*.txt")) + list(IN_DIR.rglob("*.csv"))
grand_total = grand_kept = grand_bad = 0

for fp in files:
    rel = fp.relative_to(IN_DIR)
    out_fp = OUT_DIR / rel
    out_fp = out_fp.with_suffix(".txt")
    total, kept, bad = process_file(fp, out_fp)
    grand_total += total
    grand_kept  += kept
    grand_bad   += bad
    print(f"[✔] {fp.name}: {total} lines -> {kept} kept, {bad} skipped")

print("\n Summary:")
print(f"  Files processed : {len(files)}")
print(f"  Lines seen      : {grand_total}")
print(f"  Lines kept      : {grand_kept}")
print(f"  Lines skipped   : {grand_bad}")
print(f"  Output folder   : {OUT_DIR}")
IN_DIR = Path("SAN1")
OUT_DIR = Path("SAN_processed")

def parse_timestamp(ts_raw: str):
    ts_raw = ts_raw.strip()
    fmts = [
        "%Y-%m-%d %H:%M:%S",
        "%Y/%m/%d %H:%M:%S",
        "%Y-%m-%dT%H:%M:%S",
    ]
    for f in fmts:
        try:
            return datetime.strptime(ts_raw, f)
        except ValueError:
            pass
    ts_clean = re.sub(r"[TZ]", " ", ts_raw)
    ts_clean = ts_clean.split("+")[0].split(".")[0].strip()
    for f in fmts:
        try:
            return datetime.strptime(ts_clean, f)
        except ValueError:
            pass
    raise ValueError(f"Unrecognized timestamp format: {ts_raw!r}")

def infer_vehicle_id(stem: str):
    if stem.isdigit():
        return stem
    m = re.search(r"\d+", stem)
    return m.group(0) if m else stem

def process_file(in_path: Path, out_path: Path):
    vehicle_id = infer_vehicle_id(in_path.stem)
    rows = []
    total = 0
    bad  = 0

    with in_path.open("r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            total += 1
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            parts = [p.strip() for p in line.split(",")]
            if len(parts) < 4:
                bad += 1
                continue
            try:
                lat = float(parts[0])
                lon = float(parts[1])
                ts_str = parts[-1]
                ts = parse_timestamp(ts_str)
                rows.append((ts, lon, lat))
            except Exception:
                bad += 1
                continue

    rows.sort(key=lambda x: x[0])  # sort by time ascending
    out_path.parent.mkdir(parents=True, exist_ok=True)
    with out_path.open("w", encoding="utf-8") as w:
        for ts, lon, lat in rows:
            w.write(f"{vehicle_id},{ts.strftime('%Y-%m-%d %H:%M:%S')},{lon:.6f},{lat:.6f}\n")

    return total, len(rows), bad
files = list(IN_DIR.rglob("*.txt")) + list(IN_DIR.rglob("*.csv"))
grand_total = grand_kept = grand_bad = 0

for fp in files:
    rel = fp.relative_to(IN_DIR)
    out_fp = OUT_DIR / rel
    out_fp = out_fp.with_suffix(".txt")
    total, kept, bad = process_file(fp, out_fp)
    grand_total += total
    grand_kept  += kept
    grand_bad   += bad
    print(f"[✔] {fp.name}: {total} lines -> {kept} kept, {bad} skipped")

print("\n Summary:")
print(f"  Files processed : {len(files)}")
print(f"  Lines seen      : {grand_total}")
print(f"  Lines kept      : {grand_kept}")
print(f"  Lines skipped   : {grand_bad}")
print(f"  Output folder   : {OUT_DIR}")

<Figure size 1700x700 with 0 Axes>

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54


KeyboardInterrupt: 